<h2>IUH DEVELOPER STUDENT CLUB</h2>
<h3>WORKSHOP: EXPLORING DATA WITH PYTHON AND SELENIUM</h3>

- Thời gian: 20/03/2025, từ 19h đến 21h
- Trong buổi workshop này, các bạn sẽ được tìm hiểu về Selenium và các sử dụng các hàm trong Python Selenium để thực hiện việc cào dữ liệu.

### Trước tiên, chúng ta phải thực hiện việc setup môi trường trước

Lưu ý:
- Bài viết này sử dụng môi trường Google Colab để chạy Selenium và ChromeDriver. Bạn cần cài đặt ChromeDriver và Selenium trên máy tính cá nhân nếu muốn chạy trên máy tính cá nhân. Chi tiết tham khảo file [Cheatsheet.md](Cheatsheet.md)
- Bạn cần chạy các cell dưới đây để cài đặt các thư viện cần thiết và cấu hình môi trường trước khi thực hiện các bài tập bên dưới.

In [ ]:
!apt update
!apt install -y wget unzip

# Tải Chrome for Testing phiên bản 134.0.6998.88
!wget -O google-chrome.zip "https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.88/linux64/chrome-linux64.zip"
!unzip google-chrome.zip -d /opt/google/
!ln -sf /opt/google/chrome-linux64/chrome /usr/bin/google-chrome

# Tải ChromeDriver phiên bản 134.0.6998.88
!wget -O chromedriver.zip "https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.88/linux64/chromedriver-linux64.zip"
!unzip chromedriver.zip -d /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver-linux64
!mv /usr/local/bin/chromedriver-linux64 /usr/local/bin/chromedriver


In [ ]:
!pip install selenium

In [ ]:
!ls /usr/local/bin/chromedriver

Import các thư viện cần thiết và cấu hình Chorme

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Cấu hình Chrome chạy ở chế độ headless
chrome_options = Options()
chrome_options.add_argument("--headless=new")  # Chế độ không giao diện
chrome_options.add_argument("--no-sandbox")  # Chạy trong môi trường Colab
chrome_options.add_argument("--disable-dev-shm-usage")  # Hạn chế lỗi bộ nhớ




### Bài tập 1: Cào dữ liệu từ tang web <code>genk.vn</code>

In [ ]:
# Khởi tạo WebDriver với Chrome for Testing
driver = webdriver.Chrome(service=Service("/usr/local/bin/chromedriver/chromedriver"), options=chrome_options)

# Mở trang web cần cào dữ liệu
driver.get("https://genk.vn/")

news = driver.find_elements(By.CSS_SELECTOR, '.knswli-right')

for i in range(len(news)):
  print('Title: ', news[i].find_element(By.CSS_SELECTOR, '.knswli-title').text)
  print("Subtitle: ", news[i].find_element(By.CSS_SELECTOR, '.knswli-sapo').text)
  print('Time: ', news[i].find_element(By.CSS_SELECTOR, '.knswli-meta').text)
  print('Link: ', news[i].find_element(By.CSS_SELECTOR, 'h4 > a').get_attribute('href'))
  print('\n')


# Đóng trình duyệt
driver.quit()

### Bài tập 2: Cào dữ liệu từ trang web <code>qoutes.toscrape.com</code>. Đây là trang web chuyên dụng để luyện tập việc cào dữ liệu

In [ ]:
import time
from selenium.common.exceptions import NoSuchElementException

driver1 = webdriver.Chrome(service=Service("/usr/local/bin/chromedriver/chromedriver"), options=chrome_options)
driver1.get("https://quotes.toscrape.com/")

list_name = ['sample']

while True:
    try:
        #tìm tất cả các qoutes trong 1 trang
        about_list = driver1.find_elements(By.CSS_SELECTOR, 'div.quote')
        for i in about_list:
            #với mỗi quotes, bấm vào nút 'about'
            i.find_element(By.CSS_SELECTOR, 'span > a').click()
            #hiện ra trang thông tin tác giả, ta cào cái tên trước
            name = driver1.find_element(By.CSS_SELECTOR, '.author-title').text
            check = 1
            #kiểm tra cái tên mới cào có trùng tác giả nào trong list không, có thì cho biến check = 0
            for i in list_name:
                if(name == i):
                    check = 0

            #nếu biến check = 1, nghĩa là tên vừa cào là tên mới
            if(check == 1):
                #ta thêm tên mới vào list
                list_name.append(name)

                #và in lần lượt tên, ngày sinh, nơi sinh
                print('Author name: ' + name)
                print('Date of birth: ' + driver1.find_element(By.CSS_SELECTOR, '.author-born-date').text)
                print('Born location: ' + driver1.find_element(By.CSS_SELECTOR, '.author-born-location').text)
                print('\n')
                #cho nó nghỉ 1 giây
                time.sleep(1)
                #rồi thoát ra ngoài trang chính
                driver1.back()
            #nếu check = 0 (bị trùng) thì quay về trang chính, không thực hiện thao tác cào
            else:
                driver1.back()

        #sau khi cào hết tất cả các quotes trong 1 trang, ta bấm qua trang tiếp theo
        driver1.find_element(By.CSS_SELECTOR, 'li.next > a').click()
    except NoSuchElementException:
        #nếu như đã đến trang cuối (không còn nút 'next' ở cuối trang) thì ta bắt lỗi,
        #đưa ra thông báo và thoát vòng lặp
        print('Đã đến trang cuối, thoát khỏi vòng lặp')
        break

### Bài tập 3 (về nhà): Cào dữ liệu lịch học tại trang web <code>sv.iuh.edu.vn</code> (dùng chức năng tra cứu dành cho phụ huynh)

- Khi truy cập vào trang web, ta sẽ đứng ở trang đăng nhập. Bây giờ, thay vì gõ MSSV và mật khẩu thì ta sẽ bấm vào nút "Dành cho phụ huynh".
- Sau đó chọn "Tra cứu thông tin".
- Thực hiện điền các thông tin được hiển thị trên trang web, sau đó nhấn nút "Tra cứu".
- Nó sẽ hiển thị "Kết quả tra cứu" ở phía dưới phần nhập thông tin. Ở cột "Lịch học, thi", ta chọn "Lịch theo tuần".
- Nó sẽ dẫn ta đến trang xem lịch học, bây giờ ta chỉ cần copy địa chỉ trang web (URL) và bỏ vào hàm <code>get()</code> ở dưới là xong!

In [ ]:
driver2 = webdriver.Chrome(service=Service("/usr/local/bin/chromedriver/chromedriver"), options=chrome_options)
driver2.get("Thay thế đường dẫn trang sv của bạn vào đây")

In [ ]:
table = driver2.find_element(By.CSS_SELECTOR, '.table')
date = table.find_element(By.CSS_SELECTOR, 'thead > tr')

for i in date.find_elements(By.CSS_SELECTOR, 'th'):
  if(i.text != 'Ca học'):
    print(i.text)

In [ ]:
subject = table.find_elements(By.CSS_SELECTOR, 'tbody > tr')

for i in subject:
  result = i.find_elements(By.CSS_SELECTOR, 'td')
  for j in range(1, len(result)):

    print('Subject name: ' + result[j].find_element(By.CSS_SELECTOR, 'b').text)
